In [1]:
from qiskit import IBMQ
# FIXME: first time you need input your IBM account
provider = IBMQ.load_account()

# 查看可支持的backends
provider.backends() 

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_extended_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQSimulator('simulator_stabilizer') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_manila') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_nairobi') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibm_oslo') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [8]:
from pytket.extensions.qiskit import (
    AerStateBackend,
    AerBackend,
    AerUnitaryBackend,
    IBMQBackend,
    IBMQEmulatorBackend,
)
from pytket.extensions.projectq import ProjectQBackend
from pytket import Circuit, Qubit
from pytket.utils import counts_from_shot_table
import warnings
warnings.simplefilter('ignore', DeprecationWarning)


circ = Circuit(2)
circ.H(0)
circ.CX(0, 1)

aer_state_b = AerStateBackend()
state_handle = aer_state_b.process_circuit(circ)
statevector = aer_state_b.get_result(state_handle).get_state()
print(statevector)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [9]:
aer_unitary_b = AerUnitaryBackend()
result = aer_unitary_b.run_circuit(circ)
print(result.get_unitary())

[[ 0.70710678+0.00000000e+00j  0.        +0.00000000e+00j
   0.70710678-8.65956056e-17j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j  0.70710678-8.65956056e-17j]
 [ 0.        +0.00000000e+00j  0.70710678+0.00000000e+00j
   0.        +0.00000000e+00j -0.70710678+8.65956056e-17j]
 [ 0.70710678+0.00000000e+00j  0.        +0.00000000e+00j
  -0.70710678+8.65956056e-17j  0.        +0.00000000e+00j]]


In [10]:
statevector = circ.get_statevector()
unitary = circ.get_unitary()
circ.measure_all()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [11]:
aer_b = AerBackend()
shots_handle = aer_b.process_circuit(circ, n_shots=10, seed=1)

shots = aer_b.get_result(shots_handle).get_shots()
print(shots)
print(counts_from_shot_table(shots))

[[0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
{(0, 0): 8, (1, 1): 2}


In [12]:
from qiskit.providers.aer.noise import NoiseModel
my_noise_model = NoiseModel()
readout_error = 0.2
for q in range(2):
    my_noise_model.add_readout_error(
        [[1 - readout_error, readout_error], [readout_error, 1 - readout_error]], [q]
    )

noisy_aer_b = AerBackend(my_noise_model)
noisy_shots_handle = noisy_aer_b.process_circuit(
    circ, n_shots=10, seed=1, valid_check=False
)
noisy_shots = noisy_aer_b.get_result(noisy_shots_handle).get_shots()
print(noisy_shots)

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]]


In [7]:
from pytket.utils import counts_from_shot_table
print(counts_from_shot_table(noisy_shots))

{(0, 0): 8, (1, 0): 1, (1, 1): 1}


In [8]:
backend_result = noisy_aer_b.get_result(noisy_shots_handle)
bits = circ.bits
outcomes = backend_result.get_shots([bits[1], bits[0]])
print(outcomes)

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]]


In [9]:
from pytket.backends.backendresult import BackendResult
result_dict = backend_result.to_dict()
print(result_dict)
print(BackendResult.from_dict(result_dict).get_counts())

{'qubits': [], 'bits': [['c', [1]], ['c', [0]]], 'shots': {'width': 2, 'array': [[0], [0], [0], [0], [0], [192], [0], [0], [64], [0]]}}
Counter({(0, 0): 8, (1, 0): 1, (1, 1): 1})


In [14]:
import openfermion as of
hamiltonian = 0.5 * of.QubitOperator("X0 X2") + 0.3 * of.QubitOperator("Z0")
circ2 = Circuit(3)
circ2.Y(0)
circ2.H(1)
circ2.Rx(0.3, 2)

[Y q[0]; H q[1]; Rx(0.3) q[2]; ]

In [15]:
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils.operators import QubitPauliOperator
pauli_sym = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}
def qps_from_openfermion(paulis):
    """Convert OpenFermion tensor of Paulis to pytket QubitPauliString."""
    qlist = []
    plist = []
    for q, p in paulis:
        qlist.append(Qubit(q))
        plist.append(pauli_sym[p])
    return QubitPauliString(qlist, plist)
def qpo_from_openfermion(openf_op):
    """Convert OpenFermion QubitOperator to pytket QubitPauliOperator."""
    tk_op = dict()
    for term, coeff in openf_op.terms.items():
        string = qps_from_openfermion(term)
        tk_op[string] = coeff
    return QubitPauliOperator(tk_op)
hamiltonian_op = qpo_from_openfermion(hamiltonian)

In [ ]:
from pytket.utils.operators import QubitPauliOperator
projectq_b = ProjectQBackend()
expectation = projectq_b.get_operator_expectation_value(circ2, hamiltonian_op)
print(expectation)

In [21]:
backend_name = "ibmq_manila"
# ibmq_b_emu = IBMQEmulatorBackend("ibmq_manila")
ibmq_b_emu = IBMQEmulatorBackend(backend_name)
ibmq_b_emu.valid_circuit(circ)

False

In [14]:
compiled_circ = ibmq_b_emu.get_compiled_circuit(circ)

# ibmq_b = IBMQBackend("ibmq_manila")
ibmq_b = IBMQBackend(backend_name)
ibmq_b.valid_circuit(compiled_circ)

True

In [1]:
# 测试模拟器
from pytket.extensions.qiskit import (
    AerStateBackend,
    AerBackend,
    AerUnitaryBackend,
    IBMQBackend,
    IBMQEmulatorBackend,
)

backend_name = "ibmq_qasm_simulator"
ibmq_b = IBMQBackend(backend_name)

In [15]:
quantum_handle = ibmq_b_emu.process_circuit(compiled_circ, n_shots=10)
print(ibmq_b_emu.circuit_status(quantum_handle))
quantum_shots = ibmq_b_emu.get_result(quantum_handle).get_shots()
print(quantum_shots)

CircuitStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message='job is being initialized', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)
[[0 0]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 0]]


In [16]:
circuits = []
for i in range(5):
    c = Circuit(2)
    c.Rx(0.2 * i, 0).CX(0, 1)
    c.measure_all()
    circuits.append(ibmq_b_emu.get_compiled_circuit(c))
handles = ibmq_b_emu.process_circuits(circuits, n_shots=100)
print(handles)

from pytket.utils import expectation_from_shots
for handle in handles:
    shots = ibmq_b_emu.get_result(handle).get_shots()
    exp_val = expectation_from_shots(shots)
    print(exp_val)

[ResultHandle('2eec0c48-49ad-4ef9-86ae-3ccd54e5b356', 0, 'null'), ResultHandle('2eec0c48-49ad-4ef9-86ae-3ccd54e5b356', 1, 'null'), ResultHandle('2eec0c48-49ad-4ef9-86ae-3ccd54e5b356', 2, 'null'), ResultHandle('2eec0c48-49ad-4ef9-86ae-3ccd54e5b356', 3, 'null'), ResultHandle('2eec0c48-49ad-4ef9-86ae-3ccd54e5b356', 4, 'null')]
0.92
0.92
0.94
0.8200000000000001
0.8200000000000001


In [17]:
from pytket.backends import ResultHandle
c = Circuit(2).Rx(0.5, 0).CX(0, 1).measure_all()
c = ibmq_b_emu.get_compiled_circuit(c)
handle = ibmq_b_emu.process_circuit(c, n_shots=10)
handlestring = str(handle)
print(handlestring)
# ... later ...
oldhandle = ResultHandle.from_str(handlestring)
print(ibmq_b_emu.get_result(oldhandle).get_shots())

('347481cb-0f38-48d8-bd96-ef08f336ded2', 0, 'null')
[[1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 1]
 [1 1]]
